In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Model
from keras.layers import Input,Conv2D, BatchNormalization,Activation, Add,Flatten,Dense
from keras.optimizers import Adam
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
train_data = pd.read_csv('training.csv')
test_data = pd.read_csv('test.csv')
train_data.head()
test_data.head()
train_data.ffill(inplace = True)
def load_images(data):
    # Convert the images from string format to numpy arrays
    images = []
    for img_str in data['Image']:
        try:
            img_array = np.fromstring(img_str, sep=' ')
            images.append(img_array)
        except ValueError:
            # Handle cases where the string cannot be converted
            print(f"Skipping invalid image data: {img_str[:50]}...") # Print first 50 chars for context
            pass # Skip this image

    images = np.array(images)
    images = images / 255.0  # Normalize pixel values
    images = images.reshape(-1, 96, 96, 1)  # Reshape to match Keras' input shape
    return images

In [15]:
X_train = load_images(train_data)
y_train = train_data.drop('Image', axis=1).values

X_train,X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = 42)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    tf.keras.Input(shape=(96,96,1)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(30)  # Output layer for 15 keypoints, each with x and y coordinates
])

In [16]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 94, 94, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       819,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 846,494 (3.23 MB)

 Trainable params: 846,494 (3.23 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Prepare the dataset for training using tf.data
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1024).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

# Define a custom training loop
def train_model(model, train_dataset, val_dataset, epochs):
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.MeanSquaredError()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    val_loss = tf.keras.metrics.Mean(name='val_loss')

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training loop
        for images, keypoints in train_dataset:
            with tf.GradientTape() as tape:
                predictions = model(images, training=True)
                loss = loss_fn(keypoints, predictions)

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            train_loss(loss)

        # Validation loop
        for val_images, val_keypoints in val_dataset:
            val_predictions = model(val_images, training=False)
            val_loss(loss_fn(val_keypoints, val_predictions))

        print(f"Training loss: {train_loss.result()}, Validation loss: {val_loss.result()}")
        train_loss.reset_state()
        val_loss.reset_state()

In [ ]:
# Train the model with fewer epochs for faster training
train_model(model, train_dataset, val_dataset, epochs=30)
X_test = load_images(test_data)
predictions = model.predict(X_test)
submission_format = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
submission_format.head()
# Extract individual features from predictions
predicted_keypoints = predictions.reshape(-1, 30)

# Prepare submission DataFrame
locations = []
for idx, feature in enumerate(submission_format['FeatureName']):
    image_id = submission_format.loc[idx, 'ImageId'] - 1  # ImageId is 1-based in the submission file
    feature_idx = train_data.columns.get_loc(feature)
    locations.append(predicted_keypoints[image_id, feature_idx])

submission_format['Location'] = locations
submission_format.drop(['ImageId','FeatureName'],axis =1,inplace = True)

Epoch 1/30


In [ ]:
# Save submission file
submission_format.to_csv('submission.csv', index=False)